In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torchvision

import exposenn.ontology
import exposenn.models
import exposenn.data
import exposenn.trainers

# Формирование объяснимой архитектуры с логическим блоком

Для формирования объяснимой архитектуры необходимо загрузить онтологию и определить, какие из концептов, релевантных целевому, должны фигурировать в объяснениях. Это должны быть те же самые концепты, что присутствуют в разметке.


In [2]:
# Загрузить онтологию в формате RDF/XML
# (загрузка производится с помощью owlready2, а эта библиотека полноценно
# поддерживает только RDF/XML)
ontology = exposenn.ontology.MockOntologyAnalyzer('ontologies/demo.rdf')


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp C:\Users\aaagafonov\anaconda3\envs\onto-xai\Lib\site-packages\owlready2\hermit;C:\Users\aaagafonov\anaconda3\envs\onto-xai\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/AAAGAF~1/AppData/Local/Temp/tmp6wc38csm
* Owlready2 * HermiT took 2.7379350662231445 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [3]:
# Предположим, целевым классом для задачи является класс, который в онтологии
# называется Class1.
# Используя онтологию, можно выяснить, какие классы целесообразно включать в
# онтолого-ориентированное объяснение (какие классы используются в определении
# Class1):
target_concept = ontology.get_concept('Class1')
relevant_concepts = ontology.get_relevant('Class1')

target_concept, relevant_concepts

(demo.Class1, [demo.AorB, demo.C, demo.AorB & demo.C, demo.A, demo.B])

Теоретически, все концепты из `relevant_concepts` могли бы найти свое
место в объяснимой архитектуре. Однако в нее целесообразно включать только 
такие концепты, для которых в принципе есть метки, потому что процесс обучения
(одна из его фаз или одна из используемых ЦФ) предполагает, что обучение производится с известными метками по промежуточным концептам (иначе добиться того, чтобы за определенный концепт отвечал определенный выход, по видимому, невозможно).

Стандартным форматом разметки является `csv`-файл и для него следует задать соответствие между названиями столбцов и концептами онтологии. После этого следует "проредить" полный список релевантных концептов, оставив только те, для которых есть реальные метки.

In [4]:
df = pd.read_csv('data/demo.csv')
df.head(5)

,img,A,B,classC,classD,E,F,tgt
0,001.png,1,0,1,0,0,0,1
1,002.png,1,0,1,0,0,0,1
2,003.png,1,0,1,0,0,1,1
3,004.png,1,0,1,0,0,1,1
4,005.png,1,1,1,0,0,1,1


In [5]:
df['AorB'] = df['A'] | df['B']
df.head(5)

,img,A,B,classC,classD,E,F,tgt,AorB
0,001.png,1,0,1,0,0,0,1,1
1,002.png,1,0,1,0,0,0,1,1
2,003.png,1,0,1,0,0,1,1,1
3,004.png,1,0,1,0,0,1,1,1
4,005.png,1,1,1,0,0,1,1,1


In [6]:
# df.to_csv('data/demo_extended.csv', index=False) 

In [7]:
# Настройка отображения "имя колонки"->"концепт онтологии"
column_to_concept = {
    'A': ontology.get_concept('A'),
    'B': ontology.get_concept('B'),
    'AorB': ontology.get_concept('AorB'),
    'classC': ontology.get_concept('C'),
    'tgt': ontology.get_concept('Class1')
}

Процесс настройки такого отображения может быть и, в определенном роде, двухсторонним. Например, среди релевантных концептов есть `AorB` и если мы понимаем, как он был получен из `A` и `B` то мы можем добавить вычисляемую колонку и связать ее с соответствующим концептом.

Далее, оставляем только те концепты, которые будут фигурировать в архитектуре:

In [8]:
concepts = [x for x in relevant_concepts if x in column_to_concept.values()]
concepts

[demo.AorB, demo.C, demo.A, demo.B]

In [9]:
backbone = torchvision.models.resnet18()
# Мы будем обучать задачу бинарной классификации, поэтому
# подправим количество выходов:
backbone.fc = nn.Linear(512, 1)

Сеть интерпретации может "подключаться" к разным слоям основной сети. Спецификация такого подключения описывается набором пар (слой - "коннектор"), где слой - слой базовой сети, а "коннектор" какой-то `nn.Module`, который будет на вход принимать результат работы соответствующего слоя базовой сети, а на выходе формировать тензор, попадающий на вход сети интерпретации. Для всех архитектур сетей интерпретации, которые рассматривались в ходе проекта, этот тензор должен быть одномерным (потом такие одномерные тензоры конкатенируются и "пропускаются" через набор полносвязных слоев.

In [10]:
# connectors = [
#     (backbone.layer3,  exposenn.models.GlobalAvgPool2dConnector(256)),
#     (backbone.avgpool, nn.Flatten(-3))
# ]

from exposenn.utils import create_connectors

connectors, total_features = create_connectors(backbone, [nn.Conv2d, nn.BatchNorm2d])

model = exposenn.models.MLPInterpretationNetwork(
    backbone,
    connectors,
    [total_features, len(concepts)]  
)


In [11]:
y_hat, conc_hat = model(torch.zeros((5, 3, 40, 40)))

In [12]:
conc_hat

tensor([[0.4994, 0.5020, 0.4975, 0.5009],
        [0.4994, 0.5020, 0.4975, 0.5009],
        [0.4994, 0.5020, 0.4975, 0.5009],
        [0.4994, 0.5020, 0.4975, 0.5009],
        [0.4994, 0.5020, 0.4975, 0.5009]], grad_fn=<SigmoidBackward0>)

In [13]:
y_hat.shape, conc_hat.shape

(torch.Size([5, 1]), torch.Size([5, 4]))

# Обучение модели

Предполагается поддержка двух подходов к обучению:

- обучение в рамках одного цикла со сложной функцией потерь, учитывающей разные компоненты;
- обучение на нескольких наборах с разными функциями (когда, например, прошли по образцам,
  размеченным только на целевой класс, потом по образцам, размеченным на концепты, и т.п.)


## Обучение без учета логических ограничений

Здесь важно только сопоставление целевого класса и всех меток концептов.


In [14]:
# обратное отображение концептов в колонки имеющегося набора данных
concept_to_column = {
    concept: column
    for column, concept in column_to_concept.items()
        if concept in concepts
}
# список названий колонок, которые имеет смысл использовать
# при обучении
concept_columns = list(concept_to_column.values())

In [15]:
concept_columns

['A', 'B', 'AorB', 'classC']

In [16]:
concept_to_column

{demo.A: 'A', demo.B: 'B', demo.AorB: 'AorB', demo.C: 'classC'}

In [17]:
dataset = exposenn.data.AnnotatedImagesDataset('data/demo_extended.csv',   # файл с аннотациями
                                               'data/img/',       # директория с картинками
                                               'img',             # колонка с именем файла
                                               'tgt',             # колонка с целевой меткой
                                               concept_columns,   # список с колонками-концептами
                                               transform=torchvision.transforms.ToTensor())

In [18]:
dataset[0]

(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 tensor([1, 0, 1, 1], dtype=torch.int8),
 tensor([1]))

In [19]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2)

In [20]:
from exposenn.loss import AdditiveMultipartLoss

In [21]:
optim = torch.optim.Adam(model.parameters())
loss_fn = AdditiveMultipartLoss(
    concepts_loss_fn = torch.nn.BCELoss(),
    target_loss_fn = torch.nn.BCEWithLogitsLoss()
)
history = exposenn.trainers.train(model, dataloader, loss_fn, optim, max_epochs=5)

In [22]:
history

[{'train_loss': 5.956979751586914},
 {'train_loss': 2.53704833984375},
 {'train_loss': 1.8345041275024414},
 {'train_loss': 0.9338347911834717},
 {'train_loss': 0.6312347650527954}]

## Обучение c учетом логических ограничений

In [23]:
statements = ontology.get_relevant_statements(relevant_concepts + [target_concept])
statements

[('->', demo.A, demo.AorB),
 ('->', demo.B, demo.AorB),
 ('->', demo.Class1, demo.AorB),
 ('->', demo.Class1, demo.C),
 ('->', demo.Class1, demo.AorB & demo.C)]

In [24]:
list(concept_to_column.keys())

[demo.A, demo.B, demo.AorB, demo.C]

In [25]:
from exposenn.loss import SemanticLoss

In [26]:
column_to_concept['tgt']

demo.Class1

In [27]:
# без учёта целевого класса
semantic_loss_without_tgt = SemanticLoss(list(concept_to_column.keys()), statements)

# с учётом целевого класса
semantic_loss_with_tgt = SemanticLoss(list(concept_to_column.keys()), statements, column_to_concept['tgt'])

Parsed constraint
Parsed constraint
converting to cnf
writing to DIMACS
Parsed constraint
Parsed constraint
Parsed constraint
Parsed constraint
converting to cnf
writing to DIMACS


In [28]:
semantic_loss_without_tgt(conc_hat, None)

tensor(0.4707, grad_fn=<NegBackward0>)

In [29]:
optim = torch.optim.Adam(model.parameters())
loss_fn = AdditiveMultipartLoss(
    concepts_loss_fn = semantic_loss_with_tgt,
    target_loss_fn = torch.nn.BCEWithLogitsLoss()
)
history = exposenn.trainers.train(model, dataloader, loss_fn, optim, max_epochs=5)

In [30]:
history

[{'train_loss': 2.310514450073242},
 {'train_loss': 0.8070630431175232},
 {'train_loss': 0.6123401522636414},
 {'train_loss': 0.22492147982120514},
 {'train_loss': 0.10437925159931183}]

# Два слоя: нейронная сеть и (объяснимая) логистическая регрессия

